# Agenda

1. Recap + Q&A from yesterday
2. Exercise
3. Functions
    - What are they?
    - How do we define functions?
    - Arguments and parameters
    - Return values
    - Local vs. global variables
4. Modules + packages
    - What are modules/packages?
    - How can we use them?
    - Python's standard library
    - PyPI (the Python Package Index)
    - Installing/using packages with `pip`


# Recap

1. Dictionaries
    - Dicts are key-value stores
    - There are some rules for a dict
        - Every key must be immutable
        - Keys in a dict are unique
        - Every key has a value, and every value has a key
        - There are *no* restrictions on values; they can repeat, and can be any value we want or imagine
    - We retrieve via `[]`, indicating the key whose value we want
    - We can check if a key is in a dict with `in`
        - We cannot use `in` to find if a value is in a dict
    - We can assign to a dict using `[]`
        - If the key is new, then we add a key-value pair
        - If the key exists, then we update the value for that key
    - We can iterate over a dict
        - If we iterate over the dict object, we get the keys
        - The `dict.items` method returns `(key, value)` 2-element tuples, one at a time, when we iterate. This is my favorite way to iterate over a dict
    - Three paradigms for using dicts
        - Dicts are always mutable -- but these are three conventions for working with them, that I've seen a lot of
        - Define it, retrieve from it, but never update it -- use as a small in-memory database
        - Define a dict with keys and initial values; we never add/remove keys, but we do update the values to count things
        - Define an empty dict, adding keys (as needed) and values (as needed)

2. Files
    - To work with a file, we need to use `open`, which requests help and a file object from the OS
        - When we open a file, we can specify the "mode" as the second argument to `open`, after the filename
            - `'r'` (reading, the default)
            - `'w'` (writing, which removes any previous data / zeroes out the file we open if it exists)
            - `'a'` (append, like writing, but adds to the end of a file)
    - We can read from a file in at least three ways:
        - Invoke `read()`, getting the contents, but this is considered a bit dangerous
        - Invoke `read(n)`, which returns the next `n` characters, but this is annoying, because it doesn't stop at lines.
        - Iterate over the file object, giving us one line at a time
            - Each iteration returns a string, one line in the file
            - Each line ends with `'\n'`, the line-ending character
            - When we get to the end of the file, the loop stops
    - If we want to write to a file, we use the `write` method
        - This doesn't automatically add `'\n'` to the end
    - The problem with writing is that you really need to flush + close the file, if you want to know precisely when the data is written to disk
        - You can invoke one or both of these yourself
            - `f.flush()` or `f.close()` will do these
            - You can retrieve `f.closed`, which is a `True` or `False` value, indicating if the file was closed.
        - It's common to use `with` to open a file-handling section of your code, and it automatically flushes + closes the file at the end of the block.

In [1]:
f = open('/etc/passwd')

In [2]:
f.closed

False

In [3]:
f.close()

In [4]:
f.closed

True

In [5]:
!ls *.txt

claire.txt	      mini-access-log.txt  reuven-file.txt  wcfile.txt
linux-etc-passwd.txt  nums.txt		   shoe-data.txt


# Exercise: Config writing and reading

1. Define a (small) dict with some keys and values, between 3-5 pairs.
2. Write this dict to disk in a "config file" format, meaning that each pair should be on a line by itself, with the name and value separated by `=`.
3. Then write a second program that reads the data from the file, turning each line into a key-value pair in the dict.
4. Print the resulting dict.

Example:

    # my dict
    d ={'a':10, 'b':20, 'c':30}
    
    # after my code runs, I'll see on disk:
    a=10
    b=20
    c=30
    
    # then run the second program, and I get
    {'a':10, 'b':20, 'c':30}  # or maybe values are strings

In [7]:
# Part 1: Write a dict to disk

filename = 'reuven-config.txt'
d = {'a':10, 'b':20, 'c':30}

with open(filename, 'w') as f:          # open the file for writing, and get the "with" block ready
                                        # "as f" is the "with" way of saying f=
    for key, value in d.items():        # iterate over every key-value pair in d
        f.write(f'{key}={value}\n')     # write key=value to the file on a line by itself
                                        # the with block ends, the file is flushed + closed

In [8]:
!cat reuven-config.txt

a=10
b=20
c=30


In [9]:
# try this without "with"


filename = 'reuven-config.txt'
d = {'a':10, 'b':20, 'c':30}

f = open(filename, 'w')
for key, value in d.items():        
    f.write(f'{key}={value}\n')     
f.close()   # flush + close the file                                    

# Iterating over a dict with `dict.items`

If we iterate over a dictionary, we get just the keys.

Instead of doing that, we'll invoke `dict.items` and iterate over its results. With each iteration, we'll get a 2-element tuple of `(key, value)`.



In [11]:
# this will give me, one by one, each of the key-value pairs

for something in d.items():
    print(something)

('a', 10)
('b', 20)
('c', 30)


In [12]:
# Python allows us to grab the keys and values separately, using unpacking

for key, value in d.items():   # this works so long as d.items() always returns a 2-element tuple
    print(f'{key}: {value}')

a: 10
b: 20
c: 30


In [13]:
f = open(filename, 'a')
f.write('hello=123\n')

10

In [14]:
f.close()

In [15]:
!cat $filename

a=10
b=20
c=30
hello=123


In [16]:
for apple, banana in d.items():   # this works so long as d.items() always returns a 2-element tuple
    print(f'{apple}: {banana}')

a: 10
b: 20
c: 30


In [20]:
# Reading the data

newdict = {}

for one_line in open(filename):
    new_key, new_value = one_line.split('=')
    newdict[new_key] = new_value.strip()
    
newdict    

{'a': '10', 'b': '20', 'c': '30'}

# Functions

The "DRY rule," or "don't repeat yourself," means: Don't have the same code in more than one place.

- If the same code is on several lines in a row, then we can (should) use a loop.
- If the same code is in several different places in your program, then you can use a *function*.

Functions also give us semantic power -- we can wrap up a set of things we want to do, and put them under a single name, and then refer to that whole set of tasks as one name. This is known as "abstraction," and it's a crucial idea in programming.

Functions are verbs in programming, and if we can define a function, then we can define a new verb, and enjoy the advantage of the higher level of abstraction that it gives us.

# Defining a function

I can define a new function -- that is, teach Python a new word -- using the `def` keyword:

- `def`
- followed by the name that we want to give to a function
- followed by parentheses with any parameters the function has (at first, they will be empty)
- followed by `:`
- then an indented block, known as the "function body." This can be as long or short as you want, and it can contain any code you want.

In [24]:
def hello():           # indicate the name of the function, and any parameters it might have
    print('Hello!')    # the function body -- what happens when we run the function?

I just did two things:

- created a new function object in the system
- assigned it to the name `hello` -- yes, `def` is like `=`, in that it's assigning values to variables!

This means that you cannot, in Python, have both a function and a variable named the same thing. The one that was defined/assigned later wins.

In [22]:
type(hello)

function

In [23]:
hello()  # here's how I run the function -- with ()

Hello!


In [25]:
hello()

Hello!


In [26]:
for i in range(5):
    hello()

Hello!
Hello!
Hello!
Hello!
Hello!


In [27]:
def hello():
    name = input('Enter your name: ').strip()
    print(f'Hello, {name}!')

In [28]:
hello()

Enter your name: Reuven
Hello, Reuven!


In [32]:
def count_vowels():
    text = input('Enter text: ').strip()
    count = 0
    for one_character in text:
        if one_character in 'aeiou':
            count += 1
    print(f'Number of vowels: {count}')

In [30]:
count_vowels()

Enter tex: you forgot the t at the end of the sentence
Number of vowels: 13


# Exercise: Calculator

1. Define a function, `calc`, that takes no arguments / has no parameters.
2. Inside of the function, ask the user to enter a number, an operator, and another number. (Three different things, assigned to three different variables.)
3. The operator can be either `+` or `-`.
4. Print the result of the math operation they've requested.

Example:

    calc()
    Enter first number: 10
    Enter operator: +
    Enter second number: 5
    10 + 5 = 15
    
    calc()
    Enter first number: 10
    Enter operator: -
    Enter second number: 3
    10 - 3 = 7
    
Do this in PyCharm!    


In [ ]:
def calc():
    first = input('Enter first number: ').strip()
    op = input('Enter operator: ').strip()
    second = input('Enter second number: ').strip()

    first = int(first)
    second = int(second)

    if op == '+':
        result = first + second
    elif op == '-':
        result = first - second
    else:
        result = '(Bad operator)'

    print(f'{first} {op} {second} = {result}')

calc()


This function, in the Python tutor:

https://pythontutor.com/render.html#code=def%20calc%28%29%3A%0A%20%20%20%20first%20%3D%20input%28'Enter%20first%20number%3A%20'%29.strip%28%29%0A%20%20%20%20op%20%3D%20input%28'Enter%20operator%3A%20'%29.strip%28%29%0A%20%20%20%20second%20%3D%20input%28'Enter%20second%20number%3A%20'%29.strip%28%29%0A%0A%20%20%20%20first%20%3D%20int%28first%29%0A%20%20%20%20second%20%3D%20int%28second%29%0A%0A%20%20%20%20if%20op%20%3D%3D%20'%2B'%3A%0A%20%20%20%20%20%20%20%20result%20%3D%20first%20%2B%20second%0A%20%20%20%20elif%20op%20%3D%3D%20'-'%3A%0A%20%20%20%20%20%20%20%20result%20%3D%20first%20-%20second%0A%20%20%20%20else%3A%0A%20%20%20%20%20%20%20%20result%20%3D%20'%28Bad%20operator%29'%0A%0A%20%20%20%20print%28f'%7Bfirst%7D%20%7Bop%7D%20%7Bsecond%7D%20%3D%20%7Bresult%7D'%29%0A%0Acalc%28%29&cumulative=false&curInstr=11&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=311&rawInputLstJSON=%5B%226%22,%22%2B%22,%225%22%5D&textReferences=false

# Next up

1. Arguments and parameters
2. Return values
3. Local vs. global variables

Resume at :55

In [33]:
print('abcd')

abcd


# Arguments and parameters

When we invoke a function, we very often want to pass it a value -- one that it can then use in the function. We see this all the time:

- `print` takes an argument to show
- `input` takes an argument to prompt the user
- `len` takes an argument that it should measure

For our function to take an argument, it needs a place to store that value. That's known as a "parameter," a special kind of variable that is assigned automatically when the function is invoked.

We name parameters on the top line of the function definition, inside of the `()`. You can have as many or as few parameters as you want in your function, separated by commas.

In [36]:
def hello(name):
    print(f'Hello, {name}!')

In [37]:
hello('Reuven')   

Hello, Reuven!


In [38]:
# what happens if I now call "hello" without any arguments?
hello()

TypeError: hello() missing 1 required positional argument: 'name'

In [39]:
# what happens if I call my function with an integer?
hello(5)

Hello, 5!


In [40]:
hello([10, 20, 30])

Hello, [10, 20, 30]!


In [41]:
hello({'a':10, 'b':20})

Hello, {'a': 10, 'b': 20}!


In [42]:
# I can even call my function, passing it an argument of... a function!
hello(hello)

Hello, <function hello at 0x7d40ac62af20>!


In [43]:
def add(first, second):
    print(first + second)

In [44]:
add()  # can I call it with zero arguments?

TypeError: add() missing 2 required positional arguments: 'first' and 'second'

In [45]:
add(5)

TypeError: add() missing 1 required positional argument: 'second'

In [46]:
# parameters: first second
# arguments:     5  7

add(5, 7)

12


# Exercise: Calculator

Rewrite the `calc` function from the previous exercise, such that it no longer uses `input` to ask the user for numbers or an operator. Rather, it expects to get three arguments -- an integer, a string, and an integer , and then prints the result of the math expression.

Example:

     call(10, '+', 5)
     10 + 5 = 15
     
     call(10, '*', 20)
     10 * 20 = (unknown operator)

In [49]:
def show_params(first, second, third):
    print(f'first is {first}, type is {type(first)}')
    print(f'second is {second}, type is {type(second)}')
    print(f'first is {third}, type is {type(third)}')

In [50]:
show_params(10, 'abcd', [10, 20, 30])

first is 10, type is <class 'int'>
second is abcd, type is <class 'str'>
first is [10, 20, 30], type is <class 'list'>


In [51]:
def subtract(first, second):
    print(first - second)

In [52]:
subtract(10, 3)

7


In [53]:
subtract('10', '3')

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [54]:
# solution to the above exercise

def calc(first, op, second):
    '''Prints the result of calculating the operation on first and second.

    Expects: First and second are integers, and op is either '+' or '-'
    Modifies: Prints on the screen
    Returns: (nothing)

    '''
    if op == '+':
        result = first + second
    elif op == '-':
        result = first - second
    else:
        result = '(Bad operator)'

    print(f'{first} {op} {second} = {result}')

calc(10, '+', 3)
calc(10, '+', 10)
calc(10, '-', 8)



10 + 3 = 13
10 + 10 = 20
10 - 8 = 2


# How do people know what types/values to pass?

If I'm going to call a function, then I need to know (a) how many arguments I can pass, and (b) what the function expects of me.

Where can I document this?

Answer: In the function itself!

This is a system known as "docstrings." If the first line of the function is a string (not an assignment of a string to a variable -- just a string) then that is taken as the function's documentation. The docstring is visible to anyone who hovers over the function's name.



# Comments vs. docstrings

- Comments (`#` until the end of the line) are for the person/people who will be modifying and maintaining the code.
- Docstrings are for people who will be using, or invoking, the code.

Your docstring in a function should tell people how to call the function, what the arguments should be, what it returns, and maybe a bit about what it does. It's the "what."

The comments should describe "how" and "why," and they are meant only for people who will change the function.

# Return values

We've seen that functions can *return* values:

- If I call `len('abcd')`, I get the value 4 back
- If I call `input`, I get a string back -- whatever the user typed

"Getting a value back" means that I can put the function call on the right side of an `=` operation, and give that value to a variable.

Our functions, so far, haven't returned anything useful. A function can return any value it wants with the `return` statement. A function that doesn't invoke `return` returns the special value `None` instead.

As a general rule, it's *FAR* better to return a value from a function than to print it. That's because if you return a value, the caller can decide whether to use it, assign it, print it, transform it, etc. But if you print it on the screen, the caller cannot use it.

In [55]:
def hello(name):
    return f'Hello, {name}!'

In [56]:
hello('world')

'Hello, world!'

In [57]:
result = hello('world')

In [58]:
result

'Hello, world!'

# Exercise: Calculator

Modify your `calc` function such that it doesn't print anything on the screen. Rather, it should `return` a string to the caller.

Invoke `calc` several times, and print the results that you get -- not from inside of the function, but from whoever called the function.

Example:

    result = calc(10, '+', 3)
    print(result)
    

# Keyword arguments

So far, we've called our functions using positional arguments -- meaning that the first argument is assigned to the first parameter, the second argument to the second parameter, etc.

But there is another way to associate arguments with parameters, namely "keyword arguments."

In this system, we pass arguments that look like `name=value`, with an `=` between them. The `name` must be one of the parameters defined for the function. The value will then be assigned to that parameter.

This is useful for making your function calls clearer. But it's also useful if there are a *lot* of potential arguments to a function, and you only want to pass a few.

In [59]:
def add(first, second):
    return first + second

add(10, 3)

13

In [60]:
# parameters:  first   second
# arguments:    10       3

add(first=10, second=3)   

13

In [61]:
# parameters:  first   second
# arguments:    10       3

add(second=3, first=10)   

13

In [62]:
# can I mix up positional and keyword arguments? Yes, but all of the positional arguments
# must come before all of the keyword arguments.

add(3, second=10)

13

In [63]:
add(first=3, 10)

SyntaxError: positional argument follows keyword argument (2601075463.py, line 1)

# Positional and keyword

The question is: How do arguments get assigned to parameters?

- Arguments are the values we put in `()` when we call a function
- Parameters are variables defined in the function that get assigned arguments

We normally think about positional arguments:

- The arguments that we pass are assigned to the parameters in the same order as they appear in the function definition. If we have a function `myfunc(a, b, c)`, then if I call `myfunc(10, 20, 30)`, then `a` will be assigned 10, `b` will get 20, and `c` will get 30. This is how we've been working until now.

- Another way to pass arguments is to dictate to the function which parameter should get which value. We do this with the syntax of `name=value`. If I want I can call `myfunc` as follows: `myfunc(b=10, c=20, a=30)`. In this case, the function, when called, will get the values assigned to parameters not in order, but based on the names we give. 

In [64]:
def myfunc(a, b, c):
    print(f'a = {a}, b = {b}, c = {c}')

In [65]:
myfunc(10, 20, 30)

a = 10, b = 20, c = 30


In [66]:
myfunc(b=10, c=20, a=30)

a = 30, b = 10, c = 20


In [67]:
myfunc(10, b=20, c=30)

a = 10, b = 20, c = 30


In [68]:
myfunc(10, c=20, b=30)

a = 10, b = 30, c = 20


In [69]:
myfunc(10, a=20, b=30)  

TypeError: myfunc() got multiple values for argument 'a'

In [70]:
myfunc(a=10, 20, 30)

SyntaxError: positional argument follows keyword argument (3022212719.py, line 1)

# Default argument values

You can make a parameter optional by giving it a default value. If you do that, then the caller doesn't need to pass an argument; you'll just use your default argument.

We do this by, when we define the function, not only giving a variable name for a parameter, but also `=` and then the default value. If we pass an argument, it is used. If we don't, then the default is used.

You can have as many (or as few) paramters as you want with default argument values. But they must all come after the parameters that lack default argument values. In other words: Mandatory parameters must come before optional parameters.

In [71]:
def add(first, second=3):
    return first + second

In [72]:
# parameters: first second
# arguments:    10    8

add(10, 8)

18

In [73]:
# parameters: first second
# arguments:   10    3

add(10)

13

In [74]:
s = 'a b : c d'
s.split(':')

['a b ', ' c d']

In [75]:
s.split()   # notice -- we aren't passing an argument, so there must be a default

['a', 'b', ':', 'c', 'd']

In [76]:
help(str.split)

Help on method_descriptor:

split(self, /, sep=None, maxsplit=-1) unbound builtins.str method
    Return a list of the substrings in the string, using sep as the separator string.

      sep
        The separator used to split the string.

        When set to None (the default value), will split on any whitespace
        character (including \n \r \t \f and spaces) and will discard
        empty strings from the result.
      maxsplit
        Maximum number of splits.
        -1 (the default value) means no limit.

    Splitting starts at the front of the string and works to the end.

    Note, str.split() is mainly useful for data that has been intentionally
    delimited.  With natural text that includes punctuation, consider using
    the regular expression module.



# Next up

- Modules and packages
- PyPI and `pip`

Course survey: https://app.performitiv.com/fv2/cisco/ceoevt/VC00515667

We will return at 1:30 p.m. Eastern

# DRY rule -- "don't repeat yourself"

- If you have several lines that repeat themselves in a program, use a loop.
- If you have code that repeats itself across a program, use a function.
- If you have code that repeats itself across multiple programs, use a *library*.

Every language that I know of supports libraries -- collections of variables, data structures, functions, and data types that you can use in your program without having to define them. We can use libraries to cut down on how much code we write for our own projects, share libraries with other people who have common needs/interests, and also use libraries that other people wrote to avoid reinventing the wheel.

In Python, our libraries are called "modules" or "packages":

- A module is a single file containing Python definitions
- A package is a directory containing multiple modules

But a module does more than that: Modules provide us with "namespaces," which ensure that my variables and your variables don't collide. You can think of namespaces as last names.

It's a rare program in Python that doesn't use at least one module (that someone else wrote).

1. How do we use modules?
2. Python standard library
3. PyPI (third-party modules we can download and install)
4. Demos of a few modules from PyPI


# Using modules

If I want to use code from a module, I use the Python `import` statement. For example, if I want to use the `random` module, which has functionality regarding random numbers, I can say
 
    import random

Notice:

- No parentheses! It's not a function. It's just a statement.
- The argument it gets isn't a string. It's the name of the variable we want to define that'll contain a module. (Yes, `import` is assigning to a variable, just as `def` does, and just as `=` does.)
- Python adds a `.py` to the module name, and then goes and looks for a file of that name.
- Once it's loaded, we can use `random` and `.` to retrieve names from that module.

In [77]:
import random

In [78]:
type(random)  # what is this?

module

In [79]:
# show me the printed representation for this object

random

<module 'random' from '/usr/lib/python3.12/random.py'>

In [80]:
# the random module has as "randint" function. You give it two integers, and it returns 
# a random integer from that range.

random.randint(0, 100)

76

In [81]:
random.randint(0, 100)

77

In [82]:
random.randint(0, 100)

59

# Finding what's in a module

1. We can run the `dir` function on a module, and get a list of strings -- the names of attributes we can access after the module name.  This is really a Jupyter thing.
2. We can run the `help` function on a module, getting its docstring. You can, in PyCharm, hover over the name of a module and see this.
3. We can go to the Python documentation site, and get full documentation.

In [83]:
dir(random)

['BPF',
 'LOG4',
 'NV_MAGICCONST',
 'RECIP_BPF',
 'Random',
 'SG_MAGICCONST',
 'SystemRandom',
 'TWOPI',
 '_ONE',
 '_Sequence',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_accumulate',
 '_acos',
 '_bisect',
 '_ceil',
 '_cos',
 '_e',
 '_exp',
 '_fabs',
 '_floor',
 '_index',
 '_inst',
 '_isfinite',
 '_lgamma',
 '_log',
 '_log2',
 '_os',
 '_pi',
 '_random',
 '_repeat',
 '_sha512',
 '_sin',
 '_sqrt',
 '_test',
 '_test_generator',
 '_urandom',
 '_warn',
 'betavariate',
 'binomialvariate',
 'choice',
 'choices',
 'expovariate',
 'gammavariate',
 'gauss',
 'getrandbits',
 'getstate',
 'lognormvariate',
 'normalvariate',
 'paretovariate',
 'randbytes',
 'randint',
 'random',
 'randrange',
 'sample',
 'seed',
 'setstate',
 'shuffle',
 'triangular',
 'uniform',
 'vonmisesvariate',
 'weibullvariate']

# Where does Python look for modules?

When we say `import SOMETHING`, Python looks for `SOMETHING.py` on the filesystem. Where does it look? In each of the directories in `sys.path`.

Note: `sys` is a module! So if you want to look at `sys.path`, you have to `import` it:

In [84]:
import sys
sys.path

['/usr/lib/python312.zip',
 '/usr/lib/python3.12',
 '/usr/lib/python3.12/lib-dynload',
 '',
 '/home/student/.local/lib/python3.12/site-packages',
 '/usr/local/lib/python3.12/dist-packages',
 '/usr/lib/python3/dist-packages']

This is a list of directories (and zipfiles), in which Python will look, one at a time, for the file we've mentioned. The first match wins -- this means that if there are two files with the same name in different directories, the one closer to the beginnign of `sys.path` wins.

You cannot directly load a file into Python using the import system. 

# Exercise: Guessing game

1. Select a random number from 1-100 with `random.randint`.
2. Ask the user to guess the number.
3. Once again, we'll tell them whether the guess is too high, too low, or they got it.
4. If they guess correctly, exit the loop/program.
5. If you want, you can also number the guesses, so that they can feel bad if they take a long time.

Example:

    Guess: 50
    50 is too high, guess again
    Guess: 25
    25 is too low, guess again
    Guess: hello
    hello is not numeric
    Guess: 35
    35 is right! You got it in 3 guesses.

# Other ways to use `import`

1. `import MODNAME` -- we have the module available as `MODNAME`, and its names are available under that namespace.
2. `from MODNAME import NAME` -- the module is still loaded into memory, but the only name defined is that of `NAME`, the function/data defined in `MODNAME`.

In [85]:
# what if I'm going to use randint a *lot*, and I don't want to type the whole name
# random.randint each time?

randint(0, 100)

NameError: name 'randint' is not defined

In [86]:
# I can use another syntax for import

from random import randint     # this only defines randint; it doesn't define random

In [87]:
randint(0, 100)   # this name is avilable because of "from .. import"

91

In [88]:
random.randint(0, 100)  # this is available because of "import random"

79